In [ ]:
!pip install pytorch-nlp
!pip install datasets
!pip install transformers
!pip install sentencepiece
!pip install tqdm
!pip install peft
!pip install trl
!pip install evaluate

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, AdamW
from datasets import load_dataset
from torch.utils.data import DataLoader, Dataset
import torch

from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM, GenerationConfig
from datasets import load_dataset
from peft import PeftModel, PeftConfig, LoraConfig, TaskType

# trl: Transformer Reinforcement Learning library
from trl import PPOTrainer, PPOConfig, AutoModelForSeq2SeqLMWithValueHead
from trl import create_reference_model
from trl.core import LengthSampler

import evaluate

import numpy as np
import pandas as pd

from transformers import AutoTokenizer
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead, create_reference_model
from trl.core import respond_to_batch

from tqdm import tqdm

2023-12-01 10:39:41.001518: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-01 10:39:41.001572: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-01 10:39:41.001591: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-01 10:39:41.007157: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-01 10:39:42.277964: W tensorflow/compiler/

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install rouge_score
!pip install bert_score
!pip install rouge

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [ ]:
# Load SQuAD dataset
squad_data = load_dataset("squad_v2")['train']

In [ ]:
#extracting context
c_temp = squad_data[0]['context']
#extracting answer
ans_temp = squad_data[0]['answers']['text'][0]
print(c_temp)

Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".


In [ ]:
# Prepare the dataset for fine-tuning
class SquadDataset(Dataset):
    def __init__(self, data):
        self.data = [example for example in data if example['answers']['text']]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        example = self.data[idx]
        context = example['context']
        answer = example['answers']['text'][0]  # Assuming only one answer for simplicity
        question = example['question']
        return context, "\n Question for the above generated context is : " + question

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')

# Prepare the dataset
context = SquadDataset(squad_data)
print(context.__getitem__(0))

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


('Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".', '\n Question for the above generated context is : When did Beyonce start becoming popular?')


In [ ]:
def collate_fn(batch):
    contexts, questions = zip(*batch)
    # print("Print Context",contexts)
    # print("Print Context", questions)
    inputs = tokenizer(contexts, return_tensors='pt', padding=True, truncation=True)
    labels = tokenizer(questions, return_tensors='pt', padding=True, truncation=True)
    return {**inputs, 'labels': labels['input_ids']}

In [ ]:
dataloader = DataLoader(context, batch_size=50, shuffle=True, collate_fn=collate_fn)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
device

device(type='cuda')

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("allenai/t5-small-squad2-question-generation")
model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained("allenai/t5-small-squad2-question-generation")
model_ref = AutoModelForSeq2SeqLMWithValueHead.from_pretrained("allenai/t5-small-squad2-question-generation")
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# 2. initialize trainer
ppo_config = {"batch_size": 50}
config = PPOConfig(**ppo_config)
ppo_trainer = PPOTrainer(config, model, model_ref, tokenizer, data_collator=collate_fn)

/home/djoshi12/.local/lib/python3.11/site-packages/trl/trainer/ppo_trainer.py:254: UserWarning: No dataset is provided. Make sure to set config.batch_size to the correct value before training.
  warnings.warn(


In [ ]:
#query text in batches
query_txt = "He thanked all fellow bloggers and organizations that showed support."
references = [["Who did he thank?"]]
query_tensor = tokenizer.encode(query_txt, return_tensors="pt").to(model.pretrained_model.device)

In [ ]:
generation_kwargs = {
    "top_k": 0.0, # no top-k sampling
    "top_p": 1.0, # no nucleus sampling
    "do_sample": True, # yes, we want to sample
    "max_new_tokens": 32, # specify how many tokens you want to generate at most
}


In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

reward_model = pipeline("translation", model="t5-small")

/home/djoshi12/.local/lib/python3.11/site-packages/transformers/pipelines/__init__.py:1013: UserWarning: "translation" task was used, instead of "translation_XX_to_YY", defaulting to "translation_en_to_de"
  warnings.warn(


In [ ]:
from rouge import Rouge
from bert_score import score as bert_score
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score

generation_kwargs = {
    "min_new_tokens": 1, # don't ignore the EOS token (see above)
    "top_k": 0.0, # no top-k sampling
    "top_p": 1.0, # no nucleus sampling
    "do_sample": True, # yes, we want to sample
    "pad_token_id": tokenizer.eos_token_id, # most decoder models don't have a padding token - use EOS token instead
    "max_new_tokens": 32, # specify how many tokens you want to generate at most
}
num_epochs = 4
reward_coefficients = {'bert_score': 0.97, 'rouge': 0.02, 'meteor': 0.01}
reft = []
meteor = evaluate.load("meteor")
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")
for epoch in tqdm(range(num_epochs), desc="Epochs"):
    for batch_idx, batch in tqdm(enumerate(dataloader), total=len(dataloader), desc="Batches", leave=False):
        if batch_idx <=170:
            # reward = []
            query_tensors = batch["input_ids"]
            reft.append(batch["labels"])
            response_tensors = ppo_trainer.generate([item for item in query_tensors], return_prompt=False, **generation_kwargs)
            batch["response"] = tokenizer.batch_decode(response_tensors, skip_special_tokens=True)
            batch["ref"] = [tokenizer.batch_decode(k, skip_special_tokens=True) for k in reft]
            predictions_and_references = {'predictions': batch["response"], 'references': batch["ref"][0]}
            rouge_scores = []

            # indices_with_all_padding = [idx for idx, tensor in enumerate(response_tensors) if all(element == padding_token_id for element in tensor.flatten())]

            # print("Indices with all padding tokens:", indices_with_all_padding)
            rewardLen =len(predictions_and_references["predictions"])
            # for i in range(rewardLen):
            #     prediction = predictions_and_references["predictions"][i]
            #     reference = predictions_and_references["references"][i]
            #     rouge_score = rouge.compute(predictions=[prediction], references=[[reference]]).get("rougeL")
            #     rouge_scores.append(torch.tensor(rouge_score))
            #     # print("Iteration of Rouge : ", i)

            # meteor_scores = []
            # for i in range(rewardLen):
            #     prediction = predictions_and_references["predictions"][i]
            #     reference = predictions_and_references["references"][i]
            #     meteor_score = meteor.compute(predictions=[prediction], references=[[reference]]).get("meteor")
            #     meteor_scores.append(torch.tensor(meteor_score))
            #     # print("Iteration of Meteor : ", i)

            bertscores=[]
            for i in range(rewardLen):
                prediction = predictions_and_references["predictions"][i]
                reference = predictions_and_references["references"][i]
                bertscore_score = np.mean(bertscore.compute(predictions=[prediction], references=[[reference]], lang='en', verbose=True).get("precision"))
                bertscores.append(torch.tensor(bertscore_score))

            average_scores = [(reward_coefficients['rouge'] * rouge + reward_coefficients['bert_score'] * bertscore + reward_coefficients['meteor'] * meteor).clone().detach() for rouge, meteor, bertscore in zip(rouge_scores, meteor_scores, bertscores)]

            list_of_tensors_lists = [sublist.clone().detach() for sublist in query_tensors]
            print("This is :", list_of_tensors_lists)
            print(response_tensors)
            print(len(list_of_tensors))
            print(len(response_tensors))
            print(len(meteor_scores))
            print(len(response_tensors), response_tensors[0].shape)
            print(len(list_of_tensors_lists), list_of_tensors_lists[0].shape)
            [print("Query Tensor",tensor_) for tensor_ in list_of_tensors_lists if tensor_.numel() ==0]
            [print("Response Tensor",tensor_) for tensor_ in response_tensors if tensor_.numel() ==0]
            [print(tensor_.shape) for tensor_ in response_tensors]
            train_stats = ppo_trainer.step(list_of_tensors_lists, response_tensors, bertscores)
            ppo_trainer.log_stats(train_stats, batch, reward)

[nltk_data] Downloading package wordnet to /home/djoshi12/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/djoshi12/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/djoshi12/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
Batches:   0%|          | 0/1737 [00:00<?, ?it/s]Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 4357705.57 seconds, 0.00 sentences/sec



Epochs: 100%|██████████| 4/4 [02:04<00:00, 31.15s/it]      


In [ ]:
model.save_pretrained("Reward_function_novel_latest_V3")

In [ ]:
from transformers import T5ForConditionalGeneration

model_name = "Reward_function_novel_latest_V3"
model = T5ForConditionalGeneration.from_pretrained(model_name)

Some weights of the model checkpoint at Reward_function_novel_latest_V3 were not used when initializing T5ForConditionalGeneration: ['v_head.summary.weight', 'v_head.summary.bias']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
tokenizer = T5Tokenizer.from_pretrained("allenai/t5-small-squad2-question-generation")
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
input_text = c_temp
input_ids = tokenizer.encode(input_text, return_tensors="pt")
output_ids = model.generate(input_ids)
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(f"context:{c_temp} \n question generated:{output_text}")

context:Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy". 
 question generated:<pad> What was Beyonce's debut album?


/home/djoshi12/.local/lib/python3.11/site-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
import evaluate
# Get meteor
meteor = evaluate.load("meteor")
results = meteor.compute(predictions=[output_text], references=[squad_data[0]['question']])
print(results)
print(results.get("meteor"))
meteor_score = results.get("meteor")

{'meteor': 0.136986301369863}
0.136986301369863


[nltk_data] Downloading package wordnet to /home/djoshi12/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/djoshi12/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/djoshi12/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
test_data = load_dataset("squad_v2")['validation']

In [ ]:
#test context data
tcd = []
trq=[]
for i in range(1000):
    tcd.append(test_data[i]['context'])
    trq.append(test_data[i]['question'])

In [ ]:
from datasets import load_dataset
import random

# Load SQuAD v2 validation dataset
test_data = load_dataset("squad_v2")['validation']

# Set seed for reproducibility
random.seed(4500)
# random.seed(42)
# Randomly sample 500 indices
sampled_indices = random.sample(range(len(test_data)), 1000)

# Extract context and question data for the sampled indices
tcd = [test_data[i]['context'] for i in sampled_indices]
trq = [test_data[i]['question'] for i in sampled_indices]


In [ ]:
# Get meteor

meteor = evaluate.load("meteor")
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")
meteor_res=[]
rouge_res=[]
bertscore_res=[]
for i in range(500):
    meteor_res.append(meteor.compute(predictions=[tcd[i]], references=[trq[i]]).get('meteor'))
    rouge_res.append(rouge.compute(predictions=[tcd[i]], references=[trq[i]]).get('rougeL'))
    bertscore_res.append( np.mean(bertscore.compute(predictions=[tcd[i]], references=[trq[i]], lang='en', verbose=True).get("precision")) )
#print(bertscore_res)
# print(results.get("meteor"))
# meteor_score = results.get("meteor")

[nltk_data] Downloading package wordnet to /home/djoshi12/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/djoshi12/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/djoshi12/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

...

done in 4358086.01 seconds, 0.00 sentences/sec
calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 4358086.24 seconds, 0.00 sentences/sec


In [ ]:
np.mean(rouge_res)

0.10778899822861313

In [ ]:
np.mean(bertscore_res)

0.8285505795192719

In [ ]:
np.mean(meteor_res)

0.20794459573609847